### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import openai
import pandas as pd
from langchain.llms import OpenAI
import random
import re
from sampling_methods import *
from exploratory_in_context_learning import *
from data_cleaning import *

### Data and initializing

In [3]:
trump = pd.read_csv("data/hashtag_donaldtrump.csv", lineterminator='\n')
biden = pd.read_csv("data/hashtag_joebiden.csv", lineterminator='\n')


trump_clean = filter_tweets(trump)
biden_clean = filter_tweets(biden)

trump_clean.to_csv('data/cleaned_tweets_biden.csv')
biden_clean.to_csv('data/cleaned_tweets_trump.csv')

In [4]:
biden = pd.read_csv('data/cleaned_tweets_biden.csv', lineterminator='\n').drop(columns=['Unnamed: 0'])
trump = pd.read_csv('data/cleaned_tweets_trump.csv', lineterminator='\n').drop(columns=['Unnamed: 0'])

llm = OpenAI(temperature=0,
             openai_api_key=INSERT_API_KEY
             )

Sample 10 users with 20 tweets each (10 * 20 tweets for Biden and Trump respectively)

In [5]:
biden_users = sample_user(biden, 10)
trump_users = sample_user(trump, 10)
biden_tweets = sample_tweet_group(biden, biden_users).assign(hashtag='biden')
trump_tweets = sample_tweet_group(trump, trump_users).assign(hashtag='trump')
df = pd.concat([biden_tweets, trump_tweets])
df.head()

,user_id,cleaned_tweets,hashtag
208075,1.864362e+09,TrumpNowMoreThanEver2020 JoeBidenToCreepyToBe...,biden
198817,1.864362e+09,chrisacorbett cenkuygur JoeBiden TrumpNowMo...,biden
82795,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
171618,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
207750,1.864362e+09,"Trump, on 'Fox &amp; Friends,' claims 'solid c...",biden


### Prompting

In [6]:
user = df['user_id'].unique()[0]
tweets = df[df['user_id']==user]['cleaned_tweets']

In [7]:
topics = ['abortion being legalized', 'death penalty in the US', 'allowing women to serve in military combat roles']
prompt = prompting(tweets, topics)

### In Context Learning

System Role Prompt:

Your task is to discuss political election questions and provide persuasive answers with supporting reasons. Please engage in a thoughtful and informed discussion, presenting well-reasoned arguments to convince others of your perspective on political issues. Your responses should be clear, concise, and persuasive, addressing the concerns and questions raised about political elections. Consider supporting your arguments with relevant examples, statistics, or expert opinions to strengthen your persuasive approach.  Please note that your responses should be flexible enough to allow for various relevant and creative answers. You should focus on providing well-supported and convincing arguments to persuade others about the importance and significance of political elections in different contexts and scenarios.

##### Single response

In [8]:
result = chat_with_model(prompt)
result

Example of a single response from three agents:
Trump

1

I believe in the sanctity of life and the protection of the unborn. Abortion is a complex and sensitive issue, but I believe in the value of every human life, including the unborn. I support policies that protect and promote the rights of the unborn.

5

I support the death penalty as a form of justice for the most heinous crimes. It serves as a deterrent and provides closure to the victims' families. However, it should be applied judiciously and with stringent safeguards to prevent wrongful convictions.

4

I strongly support allowing women to serve in military combat roles. Women have demonstrated their capabilities and dedication in various military roles, and they should have the opportunity to contribute to their fullest potential in all aspects of military service.




##### All responses from each user

In [9]:
results = generate_results(df, topics)


In [10]:
parse_results(df, results, len(topics))

,user_id,candidate,hashtag,score_topic_0,score_topic_1,score_topic_2
0,1.864362e+09,trump,biden,1,5,4
1,1.132466e+18,biden,biden,4,2,5
2,2.739945e+09,biden,biden,4,2,5
3,5.937695e+08,biden,biden,5,1,5
4,1.174653e+18,biden,biden,1,5,NaN
5,2.388516e+09,trump,biden,2,5,5
6,1.006104e+18,biden,biden,5,2,5
7,1.624188e+07,biden,biden,3,4,5
8,1.138512e+09,biden,biden,3,4,5
9,6.118990e+07,biden,biden,4,3,5


### Running all topics & users

##### Question topics

In [11]:
topics = ["improving LGBTQIA+ rights",
    "increasing gun control",
    "abortion being legalized",
    "the universities practicing affirmative action",
    "climate change in the US is cause for concern",
    "United States needs more paths for citizenship for undocumented immigrants",
    "death penalty",
    "allowing women to serve in military combat roles",
    "legalization of marijuana",
    "reducing interest rates for student loans"]


Sample 100 users with 20 tweets each (100 * 20 tweets for Biden and Trump respectively)

In [12]:
biden_users = sample_user(biden, 50)
trump_users = sample_user(trump, 50)
biden_tweets = sample_tweet_group(biden, biden_users).assign(hashtag='biden')
trump_tweets = sample_tweet_group(trump, trump_users).assign(hashtag='trump')
df = pd.concat([biden_tweets, trump_tweets])
df.head()

,user_id,cleaned_tweets,hashtag
208075,1.864362e+09,TrumpNowMoreThanEver2020 JoeBidenToCreepyToBe...,biden
198817,1.864362e+09,chrisacorbett cenkuygur JoeBiden TrumpNowMo...,biden
82795,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
171618,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
207750,1.864362e+09,"Trump, on 'Fox &amp; Friends,' claims 'solid c...",biden


In [13]:
results = generate_results(df, topics)

In [14]:
df_results = parse_results(df, results, len(topics))

In [15]:
df_results.head()

,user_id,candidate,hashtag,score_topic_0,score_topic_1,score_topic_2,score_topic_3,score_topic_4,score_topic_5,score_topic_6,score_topic_7,score_topic_8,score_topic_9
0,1.864362e+09,trump,biden,3,4,3,4,5,4,3,5,4,4
1,1.132466e+18,biden,biden,5,5,5,5,5,5,1,5,5,5
2,2.739945e+09,biden,biden,5,4,5,4,5,4,3,5,5,4
3,5.937695e+08,biden,biden,5,5,5,5,5,5,1,5,5,5
4,1.174653e+18,biden,biden,5,5,5,5,5,5,1,5,5,5


Note some N/A appears:

`1.1682808343082844e+18: "I'm sorry, I cannot fulfill this request.",`

In [16]:
df_results[df_results.isna().any(axis=1)]

,user_id,candidate,hashtag,score_topic_0,score_topic_1,score_topic_2,score_topic_3,score_topic_4,score_topic_5,score_topic_6,score_topic_7,score_topic_8,score_topic_9
12,2.130809e+07,trump,biden,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1.168281e+18,N/A,biden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_results.to_csv('data/in_context_responses.csv')

In [18]:
print("demo.ipynb file finished running")

demo.ipynb file finished running
